<a href="https://colab.research.google.com/github/kavithabakshi/ADNI/blob/master/ADNI_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
metadata_test = 'https://raw.githubusercontent.com/kavithabakshi/ADNI/master/ADNI_Metadata_test.csv'
metadata = 'https://raw.githubusercontent.com/kavithabakshi/ADNI/master/ADNI_Metadata.csv'
df = pd.read_csv(metadata, sep='\s*,\s*',delimiter = ',')
print(df.shape)
df_test = pd.read_csv(metadata_test, sep='\s*,\s*',delimiter = ',')
print(df_test.shape)
df_cn=df[df['researchGroup']=="CN"]
print(df_cn.shape)
df_ad=df[df['researchGroup']=="AD"]
print(df_ad.shape)
df_mci=df[df['researchGroup']=="MCI"]
print(df_mci.shape)
df.info()
df.isnull().sum()

(1599, 9)
(688, 9)
(490, 9)
(332, 9)
(777, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 9 columns):
researchGroup    1599 non-null object
subjectSex       1599 non-null object
MMSCORE          1599 non-null int64
GDTOTAL          1599 non-null int64
CDGLOBAL         1599 non-null float64
NPISCORE         1599 non-null int64
FAQTOTAL         1599 non-null int64
subjectAge       1599 non-null float64
weightKg         1599 non-null float64
dtypes: float64(3), int64(4), object(2)
memory usage: 112.5+ KB


researchGroup    0
subjectSex       0
MMSCORE          0
GDTOTAL          0
CDGLOBAL         0
NPISCORE         0
FAQTOTAL         0
subjectAge       0
weightKg         0
dtype: int64

In [2]:
print(df.columns)
y = df['researchGroup']
print(y.shape)
X = df.drop(['researchGroup'], axis=1)
print(X.shape)
X = pd.get_dummies(data = X, columns = ['subjectSex'] , prefix = ['subjectSex'] , drop_first = True)
print(X.shape, '\n', X.columns)
X.head()
y = pd.get_dummies(data = y, columns = ['researchGroup'] , prefix = ['researchGroup'] , drop_first = True)
print(y.shape, '\n', y.columns)
y.head()
print(df_test.columns)
y_test = df_test['researchGroup']
print(y_test.shape)
X_test = df_test.drop(['researchGroup'], axis=1)
print(X_test.shape)
X_test = pd.get_dummies(data = X_test, columns = ['subjectSex'] , prefix = ['subjectSex'] , drop_first = True)
print(X_test.shape, '\n', X_test.columns)
X_test.head()
y_test1 = pd.get_dummies(data = y_test, columns = ['researchGroup'] , prefix = ['researchGroup'] , drop_first = True)
#print(y_test.shape, '\n', y.columns)
#y.head()
lentest = y_test.shape
lentest1 = lentest[0] - 1
print(lentest1)
for i in range(lentest1):
  if (y_test[i] == "CN"):
    y_test[i] = 0
  elif (y_test[i] == "MCI"):
    y_test[i] = 1
  else:
    y_test[i] = 2
print(y_test.head())

Index(['researchGroup', 'subjectSex', 'MMSCORE', 'GDTOTAL', 'CDGLOBAL',
       'NPISCORE', 'FAQTOTAL', 'subjectAge', 'weightKg'],
      dtype='object')
(1599,)
(1599, 8)
(1599, 8) 
 Index(['MMSCORE', 'GDTOTAL', 'CDGLOBAL', 'NPISCORE', 'FAQTOTAL', 'subjectAge',
       'weightKg', 'subjectSex_M'],
      dtype='object')
(1599, 2) 
 Index(['['researchGroup']_CN', '['researchGroup']_MCI'], dtype='object')
Index(['researchGroup', 'subjectSex', 'MMSCORE', 'GDTOTAL', 'CDGLOBAL',
       'NPISCORE', 'FAQTOTAL', 'subjectAge', 'weightKg'],
      dtype='object')
(688,)
(688, 8)
(688, 8) 
 Index(['MMSCORE', 'GDTOTAL', 'CDGLOBAL', 'NPISCORE', 'FAQTOTAL', 'subjectAge',
       'weightKg', 'subjectSex_M'],
      dtype='object')
687
0    1
1    1
2    1
3    0
4    0
Name: researchGroup, dtype: object


In [3]:
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from tensorflow.python.layers import base
import tensorflow.contrib.slim as slim

from keras.layers import Dense, Input, Embedding, Reshape, Dropout
from keras.models import Model,Sequential
from keras.layers.merge import concatenate
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm

Using TensorFlow backend.


In [6]:
from sklearn.metrics import accuracy_score, auc, roc_curve, confusion_matrix
from sklearn.metrics import classification_report
droprate = 0.4
model = Sequential()
model.add(Dense(160, input_dim=8, activation='sigmoid'))
model.add(Dropout(droprate))
model.add(Dense(108, activation='sigmoid'))
model.add(Dropout(droprate))
#model.add(Dense(64, activation='sigmoid'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs = 75)
y_pred = model.predict(X_test)
lenPred= y_pred.shape
#y_pred = ["CN" if x[0] > 0.5 elif x[1] > 0.5 "MCI" else "AD"  for x in y_pred]
y_pred1 = [x for x in y_pred]
lenPred1 = lenPred[0] - 1
print(lenPred1)
for i in range(lenPred1):
  if (y_pred[i][0] > 0.5):
    y_pred1[i] = 0
  elif (y_pred[i][1] > 0.5):
    y_pred1[i] = 1
  else:
    y_pred1[i] = 2
score = 0
for i in range(lenPred1):
  if (y_pred1[i] == y_test[i]):
    score = score + 1
accuracy = (score/(lenPred1+1))*100
print(accuracy)
eval = model.evaluate(X_test, y_test1)
print(eval)
#dnn_score=round(accuracy_score(y_test, y_pred1) * 100,2)
#print(Title,'Dataset test accuracy_score',dnn_score)
#print(classification_report(y_test_balanced, y_pred))
 


Epoch 1/75
1599/1599 [==============================] - 1s 321us/step - loss: 0.6584 - acc: 0.5979
Epoch 2/75
1599/1599 [==============================] - 0s 72us/step - loss: 0.6456 - acc: 0.5913
Epoch 3/75
1599/1599 [==============================] - 0s 71us/step - loss: 0.6192 - acc: 0.5997
Epoch 4/75
1599/1599 [==============================] - 0s 72us/step - loss: 0.5865 - acc: 0.6323
Epoch 5/75
1599/1599 [==============================] - 0s 69us/step - loss: 0.5633 - acc: 0.6592
Epoch 6/75
1599/1599 [==============================] - 0s 92us/step - loss: 0.5328 - acc: 0.7073
Epoch 7/75
1599/1599 [==============================] - 0s 68us/step - loss: 0.5090 - acc: 0.7201
Epoch 8/75
1599/1599 [==============================] - 0s 69us/step - loss: 0.4949 - acc: 0.7392
Epoch 9/75
1599/1599 [==============================] - 0s 70us/step - loss: 0.4886 - acc: 0.7339
Epoch 10/75
1599/1599 [==============================] - 0s 75us/step - loss: 0.4727 - acc: 0.7536
Epoch 11/75
1599/1